# Implementation of Theorem 18 for the quadratic form $x^2 + 2zy^2 = 512nz^2$

# Library Importation

In [48]:
import pandas as pd
import sympy as sp
from IPython.display import display

# Class definitions and solution construction

## Diophantine Solver for $x^2 + 2zy^2 = 512nz^2$ 

In [49]:
class DiophantineSolver:
    def __init__(self, z: int, max_n: int = 50, max_solutions_per_n: int = 10):
        self.z = z
        self.max_n = max_n
        self.max_solutions_per_n = max_solutions_per_n
        self.x, self.y = sp.symbols('x y', integer=True)
        self.mod_val = 32 * self.z
        self.x_constraint = 0
        self.y_constraint = 16 * self.z
        self.results_df = pd.DataFrame(columns=["x", "y", "n"])

    def _satisfies_constraints(self, x_val, y_val):
        return (
            x_val >= 0 and
            y_val >= 0 and
            x_val % self.mod_val == self.x_constraint and
            y_val % self.mod_val == self.y_constraint
        )

    def _solve_for_n(self, n):
        rhs = 512 * n * self.z**2
        eq = sp.Eq(self.x**2 + 2 * self.z * self.y**2, rhs)
        try:
            solutions = sp.diophantine(eq)
            count = 0
            seen = set()

            for sol in solutions:
                if len(sol) != 2:
                    continue
                x_expr, y_expr = sol
                try:
                    x_val = int(x_expr)
                    y_val = int(y_expr)
                except (TypeError, ValueError):
                    continue

                if self._satisfies_constraints(x_val, y_val) and (x_val, y_val) not in seen:
                    self.results_df.loc[len(self.results_df)] = [x_val, y_val, n]
                    seen.add((x_val, y_val))
                    count += 1

                if count >= self.max_solutions_per_n:
                    break

        except Exception as e:
            print(f"Error solving for n = {n}: {e}")

    def solve(self):
        for n in range(1, self.max_n + 1):
            self._solve_for_n(n)
        return self.results_df

## Pell solver for $a^2 - 2b^2 = 2z+1$ and $u^2 - 2v^2 = -2$

In [50]:
class PellSolver:
    def __init__(self, z: int):
        self.z = z
        self.A, self.B = sp.symbols('a b', integer=True)
        self.U, self.V = sp.symbols('u v', integer=True)
        self.results = {}

    def _extract_sample_solution(self, solns):
        for sol in solns:
            if all(expr.free_symbols == set() for expr in sol):
                return tuple(abs(int(expr)) for expr in sol)
            else:
                params = set()
                for expr in sol:
                    params.update(expr.free_symbols)
                subs = {param: 1 for param in params}
                try:
                    return tuple(abs(int(expr.subs(subs))) for expr in sol)
                except Exception:
                    continue
        return None

    def solve(self):
        eq1 = sp.Eq(self.A**2 - 2 * self.B**2, 2 * self.z)
        eq2 = sp.Eq(self.U**2 - 2 * self.V**2, -2)
        self.results['eq1'] = self._extract_sample_solution(sp.diophantine(eq1))
        self.results['eq2'] = self._extract_sample_solution(sp.diophantine(eq2))
        return self.results

## Solution triple construction w.r.t Theorem 17

In [51]:
def run_full_solver():
    z = int(input("Enter z: "))
    max_n = int(input("Enter max_n: "))
    max_solutions_per_n = int(input("Enter max_solutions_per_n: "))
    iterations = int(input("Enter number of iterations for matrix multiplication: "))

    dio_solver = DiophantineSolver(z=z, max_n=max_n, max_solutions_per_n=max_solutions_per_n)
    dio_solver.solve()

    pell_solver = PellSolver(z=z)
    pell_results = pell_solver.solve()

    if not pell_results.get('eq1') or not pell_results.get('eq2'):
        print("No Pell solutions found.")
        return pd.DataFrame()

    a, b = pell_results['eq1']
    u, v = pell_results['eq2']

    T = sp.Matrix([[3, 4], [2, 3]])
    uv_vec = sp.Matrix([[u], [v]])

    denom = 32 * z
    numer_const = 16 * z

    for _ in range(iterations):
        u_n, v_n = int(uv_vec[0]), int(uv_vec[1])
        triples = []

        for _, row in dio_solver.results_df.iterrows():
            x, y, n = row['x'], row['y'], row['n']
            x_k = x / denom
            y_k = (y - numer_const) / denom

            if not (x_k.is_integer() and y_k.is_integer()):
                triples.append(None)
                continue

            x_k, y_k = int(x_k), int(y_k)

            term1 = (2 * y_k + 1) * (b * u_n + a * v_n)
            term2 = (2 * x_k)

            X = (term1 + term2) // 2
            Y = (term1 - term2) // 2
            Z = ((2 * y_k + 1) * (a * u_n + 2 * b * v_n)) // 2

            triples.append((X, Y, Z))

        col_name = f"({u_n},{v_n})"
        dio_solver.results_df[col_name] = triples
        uv_vec = T @ uv_vec

    return dio_solver.results_df

# Run Output

In [52]:
if __name__ == "__main__":
    final_df = run_full_solver()
    display(final_df)

,x,y,n,"(4,3)","(24,17)","(140,99)","(816,577)","(4756,3363)"
0,0,576,36,"(174, 174, 246)","(1014, 1014, 1434)","(5910, 5910, 8358)","(34446, 34446, 48714)","(200766, 200766, 283926)"
